# $$\text{African Institute for Mathematical Sciences}$$ 
## $$Kigali , Rwanda$$  
---
## $$\text{Abdelkader YOUNOUSSI SALEY}$$ 
## $$\text{Algebra and Cryptography}$$ 
## $$\text{Assigment 2}$$ 
$$\text{Mars 8, 2025}$$
---

## $\textbf{Exercise 1.4: Sieve of Eratosthene Algorithm}$

In [1]:
import math

def sieve_of_eratosthenes(limit):
    """
    Implements the Sieve of Eratosthenes to generate all prime numbers up to a given limit.

    :param limit: The upper bound (inclusive) up to which primes should be found.
    :return: A list of all prime numbers up to the given limit.
    """

    is_prime = [True] * (limit + 1)

    # 0 and 1 are not prime numbers, so mark them as False
    is_prime[0] = is_prime[1] = False

    #Iterate from 2 to the square root of the limit
    for i in range(2, int(limit**0.5) + 1):
        if is_prime[i]:  # If the number is still marked as prime
            # Step 4: Mark all multiples of i (starting from i^2) as non-prime
            for j in range(i * i, limit + 1, i):
                is_prime[j] = False

    primes = [x for x in range(limit + 1) if is_prime[x]]

    return primes

In [2]:
# Task1: Find the 13181999-th Prime Number

# Parameters
n = 8181999
tg_idx = 5000000 + n

# Estimate upper bound for sieve
upper_bound = int(tg_idx * math.log(tg_idx) * 1.2)

# Generate primes and find the target prime
primes = sieve_of_eratosthenes(upper_bound)
tg_prime = primes[tg_idx - 1]  # -1 because indices are zero-based

print(f"The {tg_idx}-th prime number is {tg_prime}")

The 13181999-th prime number is 240392543


In [3]:
# Task2: Count the Number of Prime Numbers Between 225 and 226.

def count_primes_in_range(lower, upper):
    # Generate primes up to `upper` using Sieve of Eratosthenes
    primes = sieve_of_eratosthenes(upper)
    # Count primes within the range [lower, upper]
    count = sum(1 for p in primes if lower <= p <= upper)
    return count

# Bounds
lower_bound = 2**25
upper_bound = 2**26

# Count primes in range
prime_count = count_primes_in_range(lower_bound, upper_bound)

print(f"The number of primes between {lower_bound} and {upper_bound} is {prime_count}")


The number of primes between 33554432 and 67108864 is 1894120


## $\textbf{Exercise 2: ElGamal cryptosystem}$

## **Task Description**:

Bob intercepts from Alice the following encrypted message, where the parameters are defined as follows:

### **Group**:
$G$ is a finite cyclic group defined over $ \mathbb{F}_p $ where:
- $ p = 123456789987654353003 $
- The generator is $ g = 123456789 $.

### **Public Keys**:
- Alice's public key is $ g_A = 52808579942366933355 $.
- Bob's public key is $ g_B = 39318628345168608817 $.

Alice and Bob agree on the following encoding scheme for the characters in the message:
- $ A = 11 $, $ B = 12 $, ..., $ Z = 36 $, space = 41, $ ' = 42 $, $. = 43 $, $ , = 44 $, $ ? = 45 $.

### **Ciphertext**:
The ciphertext consists of pairs $ (y, A) $, where $ y $ is the encrypted message and $ A $ is the public session key.


## **Goal**:
Bob needs to decrypt the message using the ElGamal decryption scheme. The task involves:
1. Computing the secret key $ b $,
2. Recovering the session key $ k $,
3. Mapping the decrypted numbers back to their corresponding characters using the provided encoding scheme.


In [4]:
# Problem parameters
p = 123456789987654353003  # Large prime number
g = 123456789  # Generator of the group
gB = 39318628345168608817  # Bob’s public key

# Ciphertext is a list of (y, A) pairs
ciphertext = [
    (83025882561049910713, 66740266984208729661),
    (117087132399404660932, 44242256035307267278),
    (67508282043396028407, 77559274822593376192),
    (60938739831689454113, 14528504156719159785),
    (5059840044561914427, 59498668430421643612),
    (92232942954165956522, 105988641027327945219),
    (97102226574752360229, 46166643538418294423)]

In [5]:
def extended_gcd(a, b):
    """
    Compute the greatest common divisor (GCD) of a and b using the extended Euclidean algorithm.
    Returns (g, x, y) such that a*x + b*y = g = gcd(a, b).
    """
    if b == 0:
        return a, 1, 0
    g, x1, y1 = extended_gcd(b, a % b)
    return g, y1, x1 - (a // b) * y1


def mod_inverse(a, mod):
    """
    Compute the modular inverse of a modulo mod.
    Returns x such that (a * x) ≡ 1 (mod mod).
    Raises an error if the modular inverse does not exist.
    """
    g, x, _ = extended_gcd(a, mod)
    if g != 1:
        raise ValueError("Modular inverse does not exist")
    return x % mod

In [6]:
def prime_factors(n):
    """
    Compute the prime factorization of n.
    Returns a list of tuples (prime, exponent).
    Example: 12 to [(2,2), (3,1)] because 12 = 2² × 3¹.
    """
    factors = []
    i = 2
    while i * i <= n:
        if n % i == 0:
            count = 0
            while n % i == 0:
                n //= i
                count += 1
            factors.append((i, count))
        i += 1
    if n > 1:
        factors.append((n, 1))
    return factors

In [7]:
def chinese_remainder(remainders, mod):
    """
    Solve a system of congruences using the Chinese Remainder Theorem.
    Given x ≡ r_i (mod m_i), find x such that x ≡ r_i (mod m_i) for all i.
    """
    n = 1  # Compute the product of all moduli
    for m in mod:
        n *= m
    x = 0  # Solution accumulator
    for r, m in zip(remainders, mod):
        mi = n // m  # Compute partial modulus
        inv = mod_inverse(mi, m)  # Compute modular inverse of mi mod m
        x += r * mi * inv  # Combine the results
    return x % n  # Ensure result is within range

The two functions, `prime_factors` and `chinese_remainder`, let us use the Pohlig-Hellman algorithm. This algorithm figures out discrete logarithms in smaller groups to find $b$.

In [8]:
def pohlig_hellman(g, h, p, q):
    """
    Solve the discrete logarithm problem h = g^x (mod p) using the Pohlig-Hellman algorithm.
    It works efficiently when q (the order of g) has small prime factors.
    """
    factors = prime_factors(q)  # Factorize q to break the problem into smaller subproblems
    dlogs = []
    mods = []  
    for factor, exp in factors:
        gi = pow(g, q // factor, p)  # Compute g^(q/factor) mod p
        hi = pow(h, q // factor, p)  # Compute h^(q/factor) mod p
        c = 1
        while c < p:  # Search for the exponent x such that gi^c ≡ hi (mod p)
            if pow(gi, c, p) == hi:
                dlogs.append(c)
                mods.append(factor)
                break
            c += 1
    return chinese_remainder(dlogs, mods)  # Solve using CRT

In [9]:
def decrypt_ciphertext(y, A, b, p):
    """
    Decrypt an ElGamal ciphertext pair (y, A) using Bob's private key b.
    """
    s = pow(y, b, p)  # Compute shared secret s = y^b mod p
    s_inv = mod_inverse(s, p)  # Compute modular inverse of s
    return (A * s_inv) % p  # Compute plaintext as (A * s⁻¹) mod p

In [10]:
# Compute Bob's private key using the Pohlig-Hellman algorithm
b = pohlig_hellman(g, gB, p, p-1)
print(f"Bob's private key: {b}")

Bob's private key: 5191


In [11]:
# Decrypt the numerical message
decrypt_num_msg = [decrypt_ciphertext(y, A, b, p) for (y, A) in ciphertext]
print(f"Decrypted numerical message: {decrypt_num_msg}")

Decrypted numerical message: [19244117112225192941, 16191522142944411631, 22224125164116222533, 15282944412628192319, 30193215411522152315, 24302941141124131541, 16252841182531282943]


In [12]:
# Mapping from number to character
def decod_num_msg(x):
    """
    Convert a decrypted number into its corresponding character(s).
    Each number is split into two-digit pairs and mapped to characters.
    """
    let = {i: chr(64 + i - 10) for i in range(11, 37)}  # Maps 11-36 to A-Z
    spe = {41: ' ', 42: "'", 43: '.', 44: ',', 45: '?'}  # Special characters mapping
    mapping = {**let, **spe}  # Merge to dict
    
    # Convert number to string
    return ''.join(mapping.get(int(str(x)[i:i+2]), '?') for i in range(0, len(str(x)), 2))

In [13]:
# Decode the numerical message into text
decoded_text = ''.join(decod_num_msg(num) for num in decrypt_num_msg)
print(f"Decoded text: {decoded_text}")

Decoded text: IN GALOIS FIELDS, FULL OF FLOWERS, PRIMITIVE ELEMENTS DANCE FOR HOURS.


## $\textbf{Exercise 3}$

For this task, we are going to implement the ElGamal cryptosystem by creating a public and private key pair. Here are the steps that we will need to follow:

1. **Generate a Prime Number $p$:**
   - The prime number $p$ must satisfy the condition:
     $$
     p = 2p_1p_2 + 1
     $$
     where $p_1$ and $p_2$ are also prime numbers, and:
     $$
     p_1 < p_2 < p^3.
     $$
   - Additionally, the number of digits in $p$ must be at least 700.

2. **Set Up the ElGamal Cryptosystem:**
   - Use the generated prime $p$ to create a public and private keys:
     - Choose a generator $g$ of the cyclic group $(\mathbb{Z}/p\mathbb{Z})^\times$.
     - Select a private key $b$, where $b$ is a random integer such that $1 \leq b < p-1$.
     - Compute the public key component $B = g^b \mod p$.
     - The public key will be $(p, g, B)$, and the private key will be $b$.

3. **Encrypt a Message:**
   - To encrypt a message:
     - The sender  chooses a random ephemeral key $a$, where $1 \leq a < p-1$.
     - Compute two values:
       - $A = g^a \mod p$
       - $y = m \cdot B^a \mod p$, where $m$ is the plaintext message.
     - The ciphertext is the pair $(y, A)$.

4. **Decrypt the Message:**
   - To decrypt the ciphertext pair $(y, A)$:
     - Compute the shared secret: $k = A^b \mod p$.
     - Find the modular inverse of $k$, denoted as $k^{-1} \mod p$.
     - Recover the plaintext message: 
       $$
       m = (y \cdot k^{-1}) \mod p.
       $$


In [ ]:
import random
import sympy
import gmpy2
import math
from math import gcd

def strong_pseudoprimality_test(N, t=10):
    """
    Strong pseudoprimality test for an odd integer N >= 3.
    Repeats the test t times independently.

    Returns "composite" if N is composite, otherwise "probably prime".
    """
    if N < 3 or N % 2 == 0:
        return "composite"
    
    # Write N-1 as 2^e * m, with m odd
    e = 0
    m = N - 1
    while m % 2 == 0:
        m //= 2
        e += 1
    
    for _ in range(t):
        # Choose random x in {2, ..., N-2}
        x = random.randint(2, N - 2)
        
        if gcd(x, N) != 1:
            return "composite"
        
        # Compute y = x^m mod N
        y = pow(x, m, N)
        
        if y == 1 or y == N - 1:
            continue
        
        # Iterate to check for -1 condition
        for _ in range(e - 1):
            y = pow(y, 2, N)
            if y == N - 1:
                break
        else:
            return "composite"
    
    return "probably prime"

This function below goes to generate a large safe prime \( p \) satisfying the form:

$$
p = 2 \cdot p_1 \cdot p_2 + 1
$$

where `p1` and `p2` are prime numbers with `p1 < p2 < p1^3`.

### It works by::

1. **Bit-Length Initialization:**  
   The function computes bounds using logarithms for the prime search.

2. **Finding `p1`:**  
   A random `p1` is selected and tested for primality.

3. **Finding `p2`:**  
   A random `p2` is selected within the range `[p1+1, p1^3]` and tested for primality.

4. **Computing `p`:**  
   `p` is calculated as `2 * p1 * p2 + 1` and tested for primality.

5. **Returning the Prime:**  
   If all conditions are met, `p` is returned as a large safe prime.


In [23]:
pip install gmpy2

error: externally-managed-environment

× This environment is externally managed
╰─> To install Python packages system-wide, try apt install
    python3-xyz, where xyz is the package you are trying to
    install.
    
    If you wish to install a non-Debian-packaged Python package,
    create a virtual environment using python3 -m venv path/to/venv.
    Then use path/to/venv/bin/python and path/to/venv/bin/pip. Make
    sure you have python3-full installed.
    
    If you wish to install a non-Debian packaged Python application,
    it may be easiest to use pipx install xyz, which will manage a
    virtual environment for you. Make sure you have pipx installed.
    
    See /usr/share/doc/python3.12/README.venv for more information.

note: If you believe this is a mistake, please contact your Python installation or OS distribution provider. You can override this, at the risk of breaking your Python installation or OS, by passing --break-system-packages.
hint: See PEP 668 for the detai

In [15]:
def generate_large_safe_prime(bit_length):
    """
    Generates a large safe prime p of the form 2*p1*p2 + 1 where p1 and p2 are primes and
    p1 < p2 < p1^3.
    """
    x = 2 ** bit_length
    ln2x = sympy.log(x, 2)
    
    # Bounds y0 and y1
    y0 = (x - ln2x) / (2 * ln2x)
    y1 = 2**(bit_length-1) - 1
    
    while True:
        # Search for p1 in [y0^(1/3), y1^(1/3)]
        p1_min = int(y0**(1/3))
        p1_max = int(y1**(1/3))
                
        if p1_max > p1_min:
            # Randomly select p1 from the range [p1_min, p1_max]
            p1 = random.randint(p1_min, p1_max)
            
            # Check if p1 is probably prime
            if strong_pseudoprimality_test(p1) == "probably prime":
                # Search for p2 in the range [p1+1, p1^3]
                p2_min = p1 + 1
                p2_max = int(p1**3)
                
                # Make sure the range for p2 is valid
                if p2_max > p2_min:
                    # Randomly select p2 from the range [p2_min, p2_max]
                    p2 = random.randint(p2_min, p2_max)
                    
                    # Check if p2 is probably prime
                    if strong_pseudoprimality_test(p2) == "probably prime":
                        # Calculate the prime number p as 2*p1*p2 + 1
                        p = 2 * p1 * p2 + 1
                        
                        # Check if p is probably prime
                        if strong_pseudoprimality_test(p) == "probably prime":
                            # Return the prime number p if all conditions are met
                            return p

In [16]:
def generate_keys(p):
    """
    Generates public and private keys for the ElGamal cryptosystem.
    """
    g = random.randint(2, p-2)  # Random generator
    x = random.randint(2, p-2)  # Private key
    h = pow(g, x, p)           # Public key
    return (g, h, x)

In [17]:
def encrypt(message, g, h, p):
    """
    Encrypts a message with the public key.
    """
    k = random.randint(2, p-2)  # Random ephemeral key
    c1 = pow(g, k, p)
    c2 = (message * pow(h, k, p)) % p
    return (c1, c2)

def decrypt(ciphertext, x, p):
    """
    Decrypts a message with the private key.
    """
    c1, c2 = ciphertext
    s = pow(c1, x, p)           # Shared secret
    s_inv = pow(s, -1, p)       # Modular inverse of s
    message = (c2 * s_inv) % p
    return message

In [ ]:
# Estimating the Bit-Length of a 700-Digit Number
digits = 70
bit_length_estimate = math.ceil(digits * math.log2(10))
print(bit_length_estimate)

2326


In [20]:
# Generate a prime with at least 700 digits
bit_length = bit_length_estimate
p = generate_large_safe_prime(bit_length)
print(f"Generated prime p: {p}")
print(f"Size of p in digits: {len(str(p))}")

OverflowError: int too large to convert to float

In [ ]:
# Generate keys
g, h, x = generate_keys(p)
print(f"Public key: (p={p}, g={g}, h={h})")
print(f"Private key: x={x}")

In [ ]:
# Encrypt a message
msg = 123456789  # Example message
ciphertext = encrypt(msg, g, h, p)
print(f"Ciphertext: {ciphertext}")

# Decrypt the message
decrypted_message = decrypt(ciphertext, x, p)
print(f"Decrypted message: {decrypted_message}")